In [31]:
import pandas as pd
import os
import re
import json
from urllib.parse import urlparse
from dotenv import load_dotenv
pd.set_option("max_colwidth", 120)
load_dotenv()



True

In [32]:
db_csv_path = os.getenv('DB_URL_CSV')

cwd = os.getcwd()
db_csv_path = os.path.join(cwd, '..', db_csv_path)
print(db_csv_path)

/home/bidon01/Documents/projects/extadel_romanian/notebooks/.././data/furniture_stores_pages.csv


In [33]:


df = pd.read_csv(db_csv_path)
df.head()

,URL
0,https://www.factorybuys.com.au/products/euro-top-mattress-king
1,https://dunlin.com.au/products/beadlight-cirrus
2,https://themodern.net.au/products/hamar-plant-stand-ash
3,https://furniturefetish.com.au/products/oslo-office-chair-white
4,https://hemisphereliving.com.au/products/


In [34]:
from urllib.parse import urlparse
url = 'https://www.toplinestore.co.uk/products/valawhi&title=Vala%20High%20Gloss%20Coffee%20Tabl...&source=http://www.toplinestore.co.uk/products/valawhi'
domain1 = urlparse(url)
domain1 

ParseResult(scheme='https', netloc='www.toplinestore.co.uk', path='/products/valawhi&title=Vala%20High%20Gloss%20Coffee%20Tabl...&source=http://www.toplinestore.co.uk/products/valawhi', params='', query='', fragment='')

In [35]:
import re 
s = url 

def get_domain(url_text:str) -> str:
    domain = urlparse(url_text).netloc
    return str(domain)

def polish_text(extracted_text: str) -> str:
    excape_words = ['...', 'title=', 'source=']
    big_regex = re.compile('|'.join(map(re.escape, excape_words)))
    text_removed_words = big_regex.sub(' ', extracted_text)
    text = re.sub(r'[-{}%=&]', ' ', text_removed_words)
    text = text.strip()
    return re.sub(' +', ' ', text)

def get_raw_after_products(url_text: str) -> list[str]:
    products_lst = re.findall(r'/products/[a-zA-Z0-9\u00C0-\u024F-{}/%=&.]+', url_text)
    products_lst = [item.split('http')[0].split('/products/')[-1] for item in set(products_lst)]
    products_lst = [polish_text(text) for text in products_lst]
    if len(products_lst) == 0:
        products_lst = ['']
    text = ' '.join(products_lst)
    return text 


def get_raw_before_products(url_text: str) -> list[str]:
    products_lst = re.findall(r'[a-zA-Z0-9\u00C0-\u024F-{}/%=&.]+/products/', 
        url_text)
    domain = urlparse(url_text).netloc + '/'     
    products_lst = [item.split(domain)[-1].split('products')[0] for item in set(products_lst)]
    products_lst = [polish_text(text) for text in products_lst]
    if len(products_lst) == 0:
        products_lst = ['']
    text = ' '.join(products_lst)
    return text 



rst = get_raw_after_products(url)
# text = get_text(rst[1])
# print(text)
# rst = set(rst)
# print(rst)
# s = rst[0][8:]
# rst = re.findall(r'(https?://\S+)', s)
rst 

# urls_lst = [ ]

# def extract_all_urls(text: str):
#     extracted_urls = re.findall(r'(https?://\S+/products)', text)
#     urls_lst.extend(extracted_urls)

#     while len(extracted_urls) > 0:
#         for 

# https://chansfurniture.com/products/copy-of-42-tennant-brand-modern-style-beatrice-bathroom-sink-vanity-tb-9433-v42-wenge-color


'valawhi valawhi Vala 20High 20Gloss 20Coffee 20Tabl'

In [36]:
df['domain'] = df.apply(lambda row: get_domain(row['URL']), axis=1)
df['raw_text_after'] = df.apply(lambda row: get_raw_after_products(row['URL']), axis=1)
df['raw_text_before'] = df.apply(lambda row: get_raw_before_products(row['URL']), axis=1)
df 

,URL,domain,raw_text_after,raw_text_before
0,https://www.factorybuys.com.au/products/euro-top-mattress-king,www.factorybuys.com.au,euro top mattress king,
1,https://dunlin.com.au/products/beadlight-cirrus,dunlin.com.au,beadlight cirrus,
2,https://themodern.net.au/products/hamar-plant-stand-ash,themodern.net.au,hamar plant stand ash,
3,https://furniturefetish.com.au/products/oslo-office-chair-white,furniturefetish.com.au,oslo office chair white,
4,https://hemisphereliving.com.au/products/,hemisphereliving.com.au,,
...,...,...,...,...
699,https://signaturefinefurniture.ca/products/jacobson-bed,signaturefinefurniture.ca,jacobson bed,
700,http://aonefurniture.in/userdata/products/no%20value,aonefurniture.in,no 20value,userdata/
701,https://furnituremama.com/products/three-door-wardrobe-mirror-2,furnituremama.com,three door wardrobe mirror 2,
702,https://gfurniture.ca/products/,gfurniture.ca,,


In [37]:
# [(idx, l) for idx, l in enumerate(df['raw_text_after'].values) if l[0]!='' and len(l)>1]

In [38]:
df.to_csv('../data/furniture_stores_pages_preprocessed.csv', 
          index_label='idx', 
        #   columns=df.columns
          )

In [39]:
#!/usr/bin/env python3
from pydantic_settings import BaseSettings

from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI, AzureOpenAI
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI

import os
from dotenv import load_dotenv

load_dotenv()


def get_chat_openai():
    chat_openai = ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"),
        temperature=0
        )
    return chat_openai

def get_llm_openai():
    llm_openai = OpenAI(
        openai_api_key=os.getenv("OPENAI_API_KEY"),
        temperature=0
        )
    return llm_openai

def get_embeddings():
    
    embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"),
                                chunk_size=10,   
        )
    return embeddings

In [19]:
import pandas as pd

from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.llm import LLMChain


prefix = """
        Extract and save the relevant entities mentioned\
        in the following query together with their properties.
        Extract furniture objects/products, properties of those products\
        and their names as given in the query.

        The input is a string derived from valide URLs. 
        Extract properties such as, the color of the product, what the product\
        has been made of, it's size, form, etc.

        Some input passages may not contain any furniture products.

        The output should be in the JSON format following the schema:
        `{{
        "furniture": "designation of the product",
        "product_property": "list of properties of the product",
        "product_name": "name of the product"
        }}`, 
        where the product property may include multiple properties or can be 
        an empty python list: [] and if         any field was not found, it should remain empty.
        """

examples = [
        {
        "query": "forna plant stand small",
        "answer": """{{"furniture": "plant stand",
        "product_property": ["small"]
        "product_name": "forna"
        }}"""
        },
        {
        "query": "valencia side table",
        "answer": """{{"furniture": "side table",
        "product_property": []
        "product_name": "valencia"
        }}"""
        },
        {
        "query": "office/desks/page/ /",
        "answer": """{{"furniture": "desk",
        "product_property": ["office"]
        "product_name": ""
        }}"""
        },
        {
        "query": "mf design sofa 3 seater sales 05",
        "answer": """{{"furniture": "sofa",
        "product_property": ["3 seater"],
        "product_name": "mf design"
        }}"""
        },
        {
        "query": "athens 3pce lounge includes 2x armless 3 seater and corner ottoman in grey storm fabric",
        "answer": """{{"furniture": ["3 seater, corner ottoman"],
        "product_property": ["armless", "grey storm fabric"],
        "product_name": "3athens 3pce lounge"
        }}"""
        },
        {
        "query": "french riviera paris 3 drawer desk rmy238",
        "answer": """{{"furniture": "desk",
        "product_property": ["3 drawer"],
        "product_name": "french rivera paris"
        }}"""
        },
        {
        "query": "childrens hammock chair",
        "answer": """{{"furniture": "hammock chair",
        "product_property": ["childrens"],
        "product_name": ""
        }}"""
        }
]
 
 

# create a example template
example_template = """
Query: {query}
{answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

suffix = """
Query: {query}
"""


# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)


def extract_furniture_products(prompt=few_shot_prompt_template):

 

    # llm_openai = get_llm_openai()  
    llm_openai = get_llm_openai()

    chain_extract = LLMChain(llm=llm_openai, 
                                        prompt=prompt,
                                        # memory=memory
                                        )
    return chain_extract

chain_extract = extract_furniture_products(prompt=few_shot_prompt_template)

In [20]:
few_shot_prompt_template

FewShotPromptTemplate(input_variables=['query'], examples=[{'query': 'forna plant stand small', 'answer': '{{"furniture": "plant stand",\n        "product_property": ["small"]\n        "product_name": "forna"\n        }}'}, {'query': 'valencia side table', 'answer': '{{"furniture": "side table",\n        "product_property": []\n        "product_name": "valencia"\n        }}'}, {'query': 'office/desks/page/ /', 'answer': '{{"furniture": "desk",\n        "product_property": ["office"]\n        "product_name": ""\n        }}'}, {'query': 'mf design sofa 3 seater sales 05', 'answer': '{{"furniture": "sofa",\n        "product_property": ["3 seater"],\n        "product_name": "mf design"\n        }}'}, {'query': 'athens 3pce lounge includes 2x armless 3 seater and corner ottoman in grey storm fabric', 'answer': '{{"furniture": ["3 seater, corner ottoman"],\n        "product_property": ["armless", "grey storm fabric"],\n        "product_name": "3athens 3pce lounge"\n        }}'}, {'query': 'f

In [21]:
result =chain_extract({"query": "folding wardrbe"})
result['text']



'{"furniture": "wardrobe",\n        "product_property": ["folding"],\n        "product_name": ""\n        }'

In [ ]:
holder = { }
cnt = 0
for idx, row in df.iterrows():
    print(idx)
    text = row['raw_text_after']
    print(text)
    
    extracted = chain_extract(text)
    print(extracted)
    holder[idx] = extracted 
    cnt += 1
    # if cnt >=4:
    #     break

In [54]:
with open("json_result01.json", "w") as fl:
    json.dump(holder, fl, indent=4)

In [57]:
with open("json_result01.json", "r") as fl:
    res = json.load(fl)


In [62]:
result = {k: json.loads(r['text']) for k, r in res.items()}
result 

{'0': {'furniture': 'mattress',
  'product_property': ['euro top', 'king'],
  'product_name': ''},
 '1': {'furniture': '',
  'product_property': [],
  'product_name': 'beadlight cirrus'},
 '2': {'furniture': 'plant stand',
  'product_property': ['ash'],
  'product_name': 'hamar'},
 '3': {'furniture': 'office chair',
  'product_property': ['white'],
  'product_name': 'oslo'},
 '4': {'furniture': '', 'product_property': [], 'product_name': ''},
 '5': {'furniture': 'pendant lamp',
  'product_property': ['larger', 'metal', 'brass'],
  'product_name': 'bridger'},
 '6': {'furniture': '', 'product_property': [], 'product_name': ''},
 '7': {'furniture': '', 'product_property': [], 'product_name': ''},
 '8': {'furniture': 'dining table',
  'product_property': [],
  'product_name': ''},
 '9': {'furniture': 'lounge', 'product_property': [], 'product_name': 'nice'},
 '10': {'furniture': ['3 seater, corner ottoman'],
  'product_property': ['armless', 'grey storm fabric'],
  'product_name': 'athens 

In [63]:
furniture = {idx: item['furniture'] for idx, item in result.items()}
property = {idx: item['product_property'] for idx, item in result.items()}
name = {idx: item['product_name'] for idx, item in result.items()}



In [66]:
df['furniture_extr'] = furniture.values()
df['property_extr'] = property.values()
df['name_extr'] = name.values()



In [70]:
df.to_csv("../data/furniture_stores_extracted.csv")

In [79]:
load_dotenv()
db_ext_csv_path = os.getenv('DB_EXTRACTED_CSV')
print(db_ext_csv_path)
db_ext_csv_path = os.path.join('../', db_ext_csv_path)
df_data_extracted = pd.read_csv(db_ext_csv_path, 
                                index_col='index',
                                usecols=['URL', 'domain', 'raw_text',
                                        'furniture_extr','property_extr','name_extr'])
df_data_extracted


data/furniture_stores_extracted.csv


ValueError: Index index invalid

In [180]:
from typing import Optional
from langchain.pydantic_v1 import BaseModel
from langchain.chains import create_extraction_chain_pydantic

llm = get_chat_openai()

# Pydantic data class
class Properties(BaseModel):
    person_name: str
    person_height: int
    person_hair_color: str
    dog_breed: Optional[str]
    dog_name: Optional[str]
        
# Extraction
chain = create_extraction_chain_pydantic(pydantic_schema=Properties, llm=llm)

# Run 
inp = """Alex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde."""
chain.run(inp)

[Properties(person_name='Alex', person_height=5, person_hair_color='blonde', dog_breed=None, dog_name=None),
 Properties(person_name='Claudia', person_height=6, person_hair_color='brunette', dog_breed=None, dog_name=None)]

In [179]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain

# Run chain
# llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
# llm = OpenAI(temperature=0,  model="gpt-3.5-turbo")
llm = get_chat_openai()
# Schema
schema = {
    "properties": {
        "person_name": {"type": "string"},
        "person_height": {"type": "integer"},
        "person_hair_color": {"type": "string"},
        "dog_name": {"type": "string"},
        "dog_breed": {"type": "string"},
    },
    "required": ["person_name", "person_height"],
}

chain = create_extraction_chain(schema, llm, prompt=)

inp = """Alex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.
Alex's dog Frosty is a labrador and likes to play hide and seek."""

chain.run(inp)

[{'person_name': 'Alex',
  'person_height': 5,
  'person_hair_color': 'blonde',
  'dog_name': 'Frosty',
  'dog_breed': 'labrador'},
 {'person_name': 'Claudia',
  'person_height': 6,
  'person_hair_color': 'brunette'}]

In [200]:
# from .settings import get_llm_openai
# from .database import vector_db_local
# from .schemas import CompaniesDetected


import pandas as pd

from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.llm import LLMChain



prefix = """
        You are a world class NLP algorithm that extracts as substrings labels or queries
        related to companies. Your mission is to detect and find in the text
        1. company name/label or part of a company label/name
        OR
        2. descirption/query for a company
        
        Retrieve exact substrings from the text, do not return any modifications, 
        just substrings from the initial text.    
        
        The result should be just one sentense.
        
        Here are some examples:    
"""

# create our examples
examples = [
    {
        "query": "Retrieve all suppliers of bosch and mercedes",
        "answer": "[bosch, mercedes]"
    }, 
    {
    "query": "Retrieve the customers and competitors of IBM", # and top French companies, founded in Paris",
    "answer": "[IBM]"
    },   
]

# create a example template
example_template = """
User: {query}
{answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

suffix = """
User: {query}
"""


# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)


def get_company_extraction_chain(prompt=few_shot_prompt_template):

    # llm_openai = get_llm_openai()  
    llm_openai = get_llm_openai()

    chain_extract = LLMChain(llm=llm_openai, 
                                        prompt=prompt,
                                        # memory=memory
                                        )
    # output = chain_extract(query)
    return chain_extract
                        
### COMPANY EXTRACTION CHAIN
company_extr_chain = get_company_extraction_chain()



In [202]:
few_shot_prompt_template

FewShotPromptTemplate(input_variables=['query'], examples=[{'query': 'Retrieve all suppliers of bosch and mercedes', 'answer': '[bosch, mercedes]'}, {'query': 'Retrieve the customers and competitors of IBM', 'answer': '[IBM]'}], example_prompt=PromptTemplate(input_variables=['answer', 'query'], template='\nUser: {query}\n{answer}\n'), suffix='\nUser: {query}\n', prefix='\n        You are a world class NLP algorithm that extracts as substrings labels or queries\n        related to companies. Your mission is to detect and find in the text\n        1. company name/label or part of a company label/name\n        OR\n        2. descirption/query for a company\n        \n        Retrieve exact substrings from the text, do not return any modifications, \n        just substrings from the initial text.    \n        \n        The result should be just one sentense.\n        \n        Here are some examples:    \n')

In [201]:
company_extr_chain("All suppliers of bosch and bmw")

{'query': 'All suppliers of bosch and bmw', 'text': '[bosch, bmw]'}